In [2]:
import gym
import numpy as np
import gym.spaces
from collections import deque

$V_{i+1}(s)= max_{a \in A} \Sigma_{s' \in S} p(s', r | s, a).(r + \gamma V_i(s'))$

In [3]:
env = gym.make('FrozenLake-v0')

In [4]:
theta = 0.00001
gamma = 1
P = env.env.P
V = np.zeros(env.env.nA*env.env.nA)

In [5]:
iteration = 0
env.reset()
while True:
    iteration += 1
    print("iteration: ", iteration)
    delta=0
    for s in P.keys():
        v = V[s]
        for a in P[s].keys():
            probs, next_state, rewards, done = map(lambda x: np.array(x), zip(*P[s][a]))   
            V[s] = np.maximum(np.sum(probs * (rewards + gamma*V[next_state])), V[s])
        delta = np.maximum(abs(V[s] - v) , delta)
        
    print("delta: ", delta)
#     print("  V: ", V)
    if delta < theta:
        break

iteration:  1
delta:  0.4444444444444444
iteration:  2
delta:  0.19753086419753085
iteration:  3
delta:  0.11522633744855966
iteration:  4
delta:  0.08677538993039678
iteration:  5
delta:  0.06078379349730256
iteration:  6
delta:  0.04776916690123739
iteration:  7
delta:  0.038581452859952736
iteration:  8
delta:  0.03222772281684605
iteration:  9
delta:  0.03080568362275682
iteration:  10
delta:  0.028831421656985823
iteration:  11
delta:  0.026867941347622132
iteration:  12
delta:  0.0256728767275399
iteration:  13
delta:  0.024344672698692477
iteration:  14
delta:  0.02297718182858391
iteration:  15
delta:  0.021616165317877745
iteration:  16
delta:  0.02041061912191322
iteration:  17
delta:  0.02031897284297582
iteration:  18
delta:  0.020063644834734384
iteration:  19
delta:  0.01964021155676321
iteration:  20
delta:  0.0191171323100508
iteration:  21
delta:  0.018534942976227042
iteration:  22
delta:  0.017919576590697983
iteration:  23
delta:  0.017288340739773034
iteration:  24

In [6]:
P[14][0]

[(0.3333333333333333, 10, 0.0, False),
 (0.3333333333333333, 13, 0.0, False),
 (0.3333333333333333, 14, 0.0, False)]

In [7]:
def action_value(s, a, V, P, board_size=8):
    probs, next_state, rewards, done = map(lambda x: np.array(x), zip(*P[s][a]))   

    if a == 0: # Left
        return 0 if s%board_size==0 else V[s-1] + np.max(rewards)
    if a == 2: # Right
        return 0 if (s+1)%board_size==0 else V[s+1]+ np.max(rewards)
    if a == 1: # Down
        return 0 if int(s/board_size)==board_size-1 else V[s+board_size] + np.max(rewards)
    if a == 3: # Up
        return 0 if int(s/board_size)==0 else V[s-board_size] + np.max(rewards)

In [8]:
def pick_action(s, V, P, board_size=8):
    best_a = -1
    max_value = -1
    for a in range(env.env.nA):
        value = action_value(s, a, V, P, board_size)
        if value > max_value:
            best_a = a
            max_value = value
    return best_a, max_value

In [9]:
V.reshape(4, -1)

array([[0.82336106, 0.82330583, 0.82326744, 0.82324791],
       [0.823378  , 0.        , 0.52929697, 0.        ],
       [0.82340452, 0.82343852, 0.76462624, 0.        ],
       [0.        , 0.88228977, 0.94114433, 0.        ]])

In [10]:
pick_action(1, V, P, 4)

(0, 0.8233610606554873)

In [11]:
actions = {0: "LEFT", 1:"DOWN", 2:"RIGHT", 3:"UP"}

In [12]:
from torch.autograd import Variable
scores = deque([], maxlen=100)
counter = 0
gamma = 1

for counter in range(1000):    
    observation = env.reset()
    
    total_reward=0
    for _ in range(20):
      action = pick_action(observation, V, P, env.env.nA)
#       print("   S: ", observation)
#       print("   A: ", actions[action[0]])
      observation, reward, done, info = env.step(action[0])
      total_reward+=reward
#       print("   New S: ", observation)
      
      
      if done:
#         print("  Done with reward: ", total_reward)
        scores.append(total_reward)
        break
#       print("-------------------")


In [13]:
np.mean(scores)

0.05

In [15]:
V.reshape(4, -1)

array([[0.82336106, 0.82330583, 0.82326744, 0.82324791],
       [0.823378  , 0.        , 0.52929697, 0.        ],
       [0.82340452, 0.82343852, 0.76462624, 0.        ],
       [0.        , 0.88228977, 0.94114433, 0.        ]])